In [1]:
%matplotlib inline
import threading
import numpy as np
from numpy.linalg import norm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import ensemble, tree, linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm

import progressbar
from importlib import reload

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, log_loss
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.utils import shuffle

In [107]:
df = pd.read_excel('data/回归/train.xlsx')
tags = df['Tags'].values
del df['Tags']

number_of_reviews = df['Total_Number_of_Reviews'].values
df['Review_Total_Positive_Word_Counts'] /= number_of_reviews / 1000

labels = df['Reviewer_Score']
del df['Reviewer_Score']

for i in range(len(tags)):
    tags[i] = " ".join(list(map(lambda x: x.strip(" '"), tags[i].strip('[] ').split(','))))

In [94]:
tags.size

80000

In [57]:
X_train = tags
vectorizer = TfidfVectorizer(stop_words='english')
tfidf = vectorizer.fit_transform(X_train).todense()
words = vectorizer.get_feature_names()

In [5]:
len(words)

504

In [81]:
best_tags = dict()
for i in range(len(words)):
    if not words[i].isdigit():
        col = np.asarray(tfidf[:, i]).flatten()
        if col.var() > 0.02:
            best_tags[words[i]] = col * 100

In [82]:
best_tags

{'business': array([ 0.        ,  0.        , 55.32668389, ...,  0.        ,
         0.        ,  0.        ]),
 'classic': array([0., 0., 0., ..., 0., 0., 0.]),
 'couple': array([ 0.        ,  0.        , 33.63399437, ..., 21.66110721,
         0.        , 33.89754097]),
 'deluxe': array([0., 0., 0., ..., 0., 0., 0.]),
 'group': array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
        51.02438483,  0.        ]),
 'night': array([27.16863418,  0.        , 39.02479032, ...,  0.        ,
         0.        ,  0.        ]),
 'solo': array([0., 0., 0., ..., 0., 0., 0.]),
 'standard': array([37.85019347,  0.        ,  0.        , ...,  0.        ,
        45.86710367,  0.        ]),
 'superior': array([ 0.        ,  0.        , 54.95933919, ..., 35.39514592,
         0.        , 55.38998525]),
 'traveler': array([0., 0., 0., ..., 0., 0., 0.]),
 'twin': array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
        38.81435481, 46.36835379])}

In [83]:
df2 = pd.concat([df, pd.DataFrame(best_tags)], axis=1)

In [103]:
df[:5]

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given
0,255,8.1,57,1290,77.519380,1
1,211,8.6,0,2923,65.001711,12
2,189,9.2,27,781,371.318822,24
3,1258,9.4,20,4204,73.739296,3
4,289,8.8,0,1519,19.749835,9


In [108]:
X_train = df.iloc[:].values
y_train = labels.values

In [109]:
X_train.shape, y_train.shape

((80000, 6), (80000,))

In [49]:
import decision_tree.decision_tree
reload(decision_tree.decision_tree)
from decision_tree.decision_tree import RegressionTree

import gradient_boosting_decision_tree.gbdt_model
reload(gradient_boosting_decision_tree.gbdt_model)
from gradient_boosting_decision_tree.gbdt_model import GBDTRegressor

In [50]:
k = 5
sss = KFold(n_splits=k, shuffle=True, random_state=0)

def acc_score(y_val, y_pred):
    return np.corrcoef(y_val, y_pred)[0,1]

def cross_val(x,y,n_estimators=100,learning_rate=0.2,max_depth=3):
    tol_acc = 0
    
    for train_index, test_index in sss.split(x, y):
#         print(train_index)
#         print(test_index)
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
        reg = GBDTRegressor(n_estimators=n_estimators,learning_rate=learning_rate,max_depth=max_depth)
#         reg = GradientBoostingRegressor(n_estimators=n_estimators,learning_rate=learning_rate,max_depth=max_depth)
    
        name = reg.__class__.__name__
        reg.fit(x_train, y_train)
        y_pred = reg.predict(x_val)
        acc = acc_score(y_val, y_pred)
        print(acc)
        tol_acc += acc
        
    ave_acc = tol_acc/k
    return ave_acc


In [52]:
cross_val(X_train,y_train,n_estimators=100,learning_rate=0.15,max_depth=3)   # tag var 0.017

0.6509339134062824
0.6579967308968074
0.6544362258012215
0.6510831339580133
0.664991501446876


0.6558883011018402

In [54]:
cross_val(X_train,y_train,n_estimators=150,learning_rate=0.15,max_depth=3)   # tag var 0.017

0.6511415983932989
0.6580761217053761
0.6547010205568369
0.6508927172576224
0.6647713197573478


0.6559165555340964

In [87]:
cross_val(X_train,y_train,n_estimators=100,learning_rate=0.15,max_depth=3)   # tag var 0.02

0.6507112758860804
0.6572848815053276
0.653676667749473
0.6511148601827342
0.6651529522739688


0.6555881275195168

In [92]:
cross_val(X_train,y_train,n_estimators=100,learning_rate=0.15,max_depth=3)   # tag var 0

0.6481159691789526
0.6562297116494902
0.6516970866896992
0.6490775854000314
0.663029682341286


0.6536300070518919

In [ ]:
cross_val(X_train,y_train,n_estimators=100,learning_rate=0.15,max_depth=3)   # tag var 0

0.642408055086571
